## Load Packages

In [4]:
using JuMP;
using HiGHS;

João is a farmer from a small town who specializes in growing wheat, corn and sugar beet. He owns $500 km^2$ of land and must decide the amount of land to be allocated to each of the crops. João faces several restrictions regarding his planting. First, he must have at least $200 T$ of wheat and $240 T$ of corn to feed his cattle. Such quantities can be obtained through own plantation or by buying from the city's cooperative. The purchase prices per ton of wheat are $238 R\$/T$ and per ton of corn $210 R\$/T$. On the other hand, any excess produced in relation to the minimum can be sold at the cooperative, however with a $40\%$ discount on the purchase price ($170 R\$/T$ for wheat and $150 R\$/T$ for corn) per the cooperative's margin account. Another important restriction concerns the sale of sugar beet. By law, the sale price of a ton of beet at the cooperative is fixed at $36 R\$/T$ for the first $6000 T$ sold. After this amount, the sale price becomes $10 R\$/T$. In addition, the cooperative where João trades his products does not have sugar beet for purchase.

Let's assume that the planting cost of each crop is: $150 R\$/km^2$ for wheat, $230 R\$/km^2$ for corn, and $260 R\$/km^2$ for sugar beet. The uncertainty of the problem lies in the productivity of the land. João does not know a priori how much each $km^2$ of land will yield in tons of culture. Now assume that three scenarios of equal probabilities of occurrence were sampled: "good", "average" and "poor". In each of the states, the yield of each crop is given by:
"Bad" state: Wheat = $2 T/km^2$; Corn = $2.4 T/km^2$; Beetroot = $16 T/km^2$;
"Average" state: Wheat = $2.5 T/km^2$; Corn = $3.0 T/km^2$; Beetroot = $20 T/km^2$;
"Good" state: Wheat = $3 T/km^2$; Corn = $3.6 T/km^2$; Beetroot = $24 T/km^2$;

## 1) Formulate the first and second stage problem. Additionally, considering the probability distribution defined above, formulate and solve the deterministic equivalent problem.
## Compute the value of the stochastic solution.
## Compute the expected value of perfect information.

$$
\begin{aligned}
    &\min && 150a_1 + 230a_2 + 260a_3 + \mathbb{E}_s[Q(a,s)]  \\
    & st && a_1+a_2+a_3 = 500\\
        &&& 0 \leq a_1, a_2, a_3 \\
\end{aligned}
$$

$$
\begin{aligned}
   Q(a,s) = &\min && 238c_1 + 210c_2 - 170v_1 - 150v_2 - l_3   \\
    & st && p_1 + c_1 - v_1 \geq 200\\
        &&& p_2 + c_2 - v_2 \geq 240\\
        &&& l_3 \leq 36p_3\\
        &&& l_3 \leq 10(p_3-6000) + 36*6000\\
        &&& p_1 = a_1s_1\\
        &&& p_2 = a_2s_2\\
        &&& p_3 = a_3s_3\\
        &&& 0 \leq p_1, p_2, p_3\\
        &&& 0 \leq c_1, c_2, v_1, v_2\\
        &&& 0 \leq l_3
\end{aligned}
$$

$$
\begin{aligned}
    &\min && 150a_1 + 230a_2 + 260a_3 + \Sigma_{i=1}^3 (238c^i_1 + 210c^i_2 - 170v^i_1 - 150v^i_2 - l^i_3)\\
    & st && a_1+a_2+a_3 = 500\\
        &&& p^i_1 + c^i_1 - v^i_1 \geq 200 & i=1,2,3 \\
        &&& p^i_2 + c^i_2 - v^i_2 \geq 240 & i=1,2,3 \\
        &&& l^i_3 \leq 36p^i_3 & i=1,2,3 \\
        &&& l^i_3 \leq 10(p^i_3-6000) + 36*6000 & i=1,2,3 \\
        &&& p^i_1 = a_1s^i_1 & i=1,2,3 \\
        &&& p^i_2 = a_2s^i_2 & i=1,2,3 \\
        &&& p^i_3 = a_3s^i_3 & i=1,2,3 \\
        &&& 0 \leq p^i_1, p^i_2, p^i_3 & i=1,2,3 \\
        &&& 0 \leq c^i_1, c^i_2, v^i_1, v^i_2 & i=1,2,3 \\
        &&& 0 \leq l^i_3 & i=1,2,3 \\
        &&& 0 \leq a_1, a_2, a_3 \\
\end{aligned}
$$

In [44]:
function farmer_optimal()
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    
    rate = [2 2.5 3
            2.4 3 3.6
            16 20 24]
    
    @variable(model, 0<=a[1:3])
    @variable(model, 0<=c[1:2,1:3])
    @variable(model, 0<=v[1:2,1:3])
    @variable(model, 0<=l[1:3])
    @expression(model,p[i=1:3,j=1:3], a[i]*rate[i,j])
    @constraint(model, sum(a) <= 500)
    @constraint(model, [i=1:3], p[1,i] + c[1,i] - v[1,i] >= 200 )
    @constraint(model, [i=1:3], p[2,i] + c[2,i] - v[2,i] >= 240 )
    @constraint(model, [i=1:3], l[i] <= 36*p[3,i] )
    @constraint(model, [i=1:3], l[i] <= 10*(p[3,i]-6000) + 36*6000)
    @objective(model, Min, 150a[1] + 230a[2] + 260a[3] + (1/3)*sum(238*c[1,:]) + (1/3)*sum(210*c[2,:]) - (1/3)*sum(170*v[1,:]) - (1/3)*sum(150*v[2,:]) - (1/3)*sum(l))

    optimize!(model)
    return model
end

function farmer_crystal_ball()
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    
    rate = [2 2.5 3
            2.4 3 3.6
            16 20 24]
    
    @variable(model, 0<=a[1:3,1:3])
    @variable(model, 0<=c[1:2,1:3])
    @variable(model, 0<=v[1:2,1:3])
    @variable(model, 0<=l[1:3])
    @expression(model,p[i=1:3,j=1:3], a[i,j]*rate[i,j])
    @constraint(model,[j = 1:3], sum(a[:,j]) <= 500)
    @constraint(model, [i=1:3], p[1,i] + c[1,i] - v[1,i] >= 200 )
    @constraint(model, [i=1:3], p[2,i] + c[2,i] - v[2,i] >= 240 )
    @constraint(model, [i=1:3], l[i] <= 36*p[3,i] )
    @constraint(model, [i=1:3], l[i] <= 10*(p[3,i]-6000) + 36*6000)
    @objective(model, Min, (1/3)*sum(150*a[1,:]) + (1/3)*sum(230*a[2,:]) + (1/3)*sum(260*a[3,:]) + (1/3)*sum(238*c[1,:]) + (1/3)*sum(210*c[2,:]) - (1/3)*sum(170*v[1,:]) - (1/3)*sum(150*v[2,:]) - (1/3)*sum(l))

    optimize!(model)
    return model
end

function farmer_average()
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    rate = [2 2.5 3
            2.4 3 3.6
            16 20 24]
    rate_average = sum(rate, dims=2)/3
    
    @variable(model, 0<=a[1:3])
    @variable(model, 0<=c[1:2])
    @variable(model, 0<=v[1:2])
    @variable(model, 0<=l)
    @expression(model,p[i=1:3], a[i]*rate_average[i])
    @constraint(model, sum(a) <= 500)
    @constraint(model, p[1] + c[1] - v[1] >= 200 )
    @constraint(model, p[2] + c[2] - v[2] >= 240 )
    @constraint(model, l <= 36*p[3] )
    @constraint(model, l <= 10*(p[3]-6000) + 36*6000)
    @objective(model, Min, 150a[1] + 230a[2] + 260a[3] + 238*c[1] + 210*c[2] - 170*v[1] - 150*v[2] - l)
    optimize!(model)
    a_opt = value.(a)
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    @variable(model, a_opt[i]<=a[i=1:3]<=a_opt[i])
    @variable(model, 0<=c[1:2,1:3])
    @variable(model, 0<=v[1:2,1:3])
    @variable(model, 0<=l[1:3])
    @expression(model,p[i=1:3,j=1:3], a[i]*rate[i,j])
    @constraint(model, sum(a) <= 500)
    @constraint(model, [i=1:3], p[1,i] + c[1,i] - v[1,i] >= 200 )
    @constraint(model, [i=1:3], p[2,i] + c[2,i] - v[2,i] >= 240 )
    @constraint(model, [i=1:3], l[i] <= 36*p[3,i] )
    @constraint(model, [i=1:3], l[i] <= 10*(p[3,i]-6000) + 36*6000)
    @objective(model, Min, 150a[1] + 230a[2] + 260a[3] + (1/3)*sum(238*c[1,:]) + (1/3)*sum(210*c[2,:]) - (1/3)*sum(170*v[1,:]) - (1/3)*sum(150*v[2,:]) - (1/3)*sum(l))

    optimize!(model)
    
    return model
end

farmer_average (generic function with 1 method)

In [48]:
model_1 = farmer_optimal()
model_2 = farmer_crystal_ball()
model_3 = farmer_average()
obj1 = objective_value(model_1)
obj2 = objective_value(model_2)
obj3 = objective_value(model_3)
@show a_1 = value.(model_1[:a])
@show a_2 = value.(model_2[:a])
@show a_3 = value.(model_3[:a])

println("Optimal Solution = $(obj1)")
println("Wait and See Solution = $(obj2)")
println("Average Solution = $(obj3)")
println("VSS = $(obj1-obj2)")
println("EPVI = $(obj3-obj1)")

a_1 = value.(model_1[:a]) = [170.0, 80.0, 250.0]
a_2 = value.(model_2[:a]) = [100.0 120.0 183.33333333333331; 25.0 80.0 66.66666666666667; 375.0 300.0 250.0]
a_3 = value.(model_3[:a]) = [120.0, 80.0, 300.0]
Optimal Solution = -108390.0
Wait and See Solution = -115405.55555555556
Average Solution = -107240.0
VSS = 7015.555555555562
EPVI = 1150.0
